In [ ]:
# MNIST 데이터셋을 로드하고 CNN을 정의하여 모델을 학습시키고
# 테스트 데이터에 대한 예측 및 정확도 평가까지 수행하는 전체 딥러닝 파이프라인

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# 데이터 전처리 및 시각화를 위해 사용

import torch
import torch.nn as nn
import torch.optim as optim
# PyTorch의 딥러닝 관련 모듈

from tqdm.notebook import tqdm_notebook
# Colab에서 학습 progress bar 표시

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# 구글 드라이브의 데이터를 사용하기 위한 마운트

In [ ]:
def show(img): ##visualize your image
    plt.imshow(img[1:].reshape(28, 28), cmap='gray')
    # 첫 번째 컬럼(label) 제외 → 픽셀만 사용, (28,28)로 reshape하여 이미지로 보여줌
    plt.show()

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/mnist_train.csv')
pxl = ["pxl{}".format(i) for i in range(len(df.columns)-1)]
df.columns = ["value"] + pxl
#df.rename(columns = {"5":"value"}, inplace = True)
# MNIST 학습 데이터를 읽고 컬럼 이름을 숫자(label)와 픽셀값들로 나눔

In [ ]:
df.head(10)
# 첫 10개 데이터 출력

In [ ]:
show(df.iloc[3].values) #visualize 3rd dataset (hand writing image)
# 3번째 숫자 이미지 시각화

In [ ]:
A = pd.DataFrame(df, columns = pxl).values
# A: 이미지의 픽셀 값만 추출
y_data = pd.DataFrame(df, columns = ["value"]).values
# y_data: 정답 숫자(0~9)

In [ ]:
A.shape

In [ ]:
def digit_to_vec(value):
  vec = np.zeros(shape = 10)
  vec[value] = 1
  return vec
# 라벨을 원-핫 벡터로 변환
# ex) 5 → [0,0,0,0,0,1,0,0,0,0]로 변환

In [ ]:
y_data_vec = [digit_to_vec(y_data[k]) for k in range(len(y_data))]
B = np.array(y_data_vec)

In [ ]:
B

In [ ]:
x = torch.tensor(A, dtype=torch.float32)
y = torch.tensor(B, dtype=torch.float32)
x = x.unsqueeze(1) #new dimension at 1st comp.
y = y

x = x.view(-1,1,28,28)
# Torch Tensor로 변환

In [ ]:
x.shape

In [ ]:
x[0].shape

In [ ]:
conv = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=(3,3)) #previous: conv1d, kernel_size=3
# CNN 개념 테스트

In [ ]:
list(conv.parameters())

In [ ]:
plt.imshow(x[3].detach().numpy().squeeze(0))

In [ ]:
conv(x[0]).size()

In [ ]:
conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(3,3))
# Conv2D + MaxPooling이 어떻게 작동하는지 확인

In [ ]:
x1 = conv(x[0])

In [ ]:
print(x1.shape)

In [ ]:
pool = nn.MaxPool2d(kernel_size=(3,3))

In [ ]:
list(pool.parameters())

In [ ]:
pool(x1).shape

In [ ]:
x2 = pool(x1)
print(x2.shape)

In [ ]:
class CNN(nn.Module):
  # CNN 모델 정의
  # conv1, conv2: 합성곱 계층
  # pool: 최대 풀링, linear: 마지막 분류층, softmax: 확률로 출력
  # status=True: 중간 출력 shape 확인용
    def __init__(self, status):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=(3,3))
        self.conv2 = nn.Conv2d(in_channels=2, out_channels=3, kernel_size=(3,3))

        self.linear = nn.Linear(75,10)
        self.pool = nn.MaxPool2d(kernel_size=(2,2))

        self.softmax = nn.Softmax(dim=1)
        self.status = status

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(x)
        if self.status:
          print(x.shape)

        x = self.conv2(x)
        x = self.pool(x)
        if self.status:
          print(x.shape)

        x = x.view(x.size(0), -1)
        if self.status:
          print(x.shape)
        x = self.linear(x)
        x = self.softmax(x)
        if self.status:
          print(x.shape)
        return x

In [ ]:
model = CNN(status=True)
# 모델 생성 및 출력 확인

In [ ]:
model(x[:2])

In [ ]:
y-model(x)
# 의미 없는 코드

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-5)
# Adam Optimizer, MSE 손실 함수 사용

loss_fn = nn.MSELoss()

num_epochs = 1000
# 전체 데이터를 배치 사이즈 20으로 1000번 학습


In [ ]:
if True:

  batch_size = 20
  num_batches = x.size(0) // batch_size

  for epoch in range(num_epochs):
    # 미니배치 학습
      model.train()
      # 학습 모드 설정
      # optimizer.zero_grad() → loss.backward() → optimizer.step(): 학습 단계

      for i in tqdm_notebook(range(0, x.size(0), batch_size)):
          x_batch = x[i:i+batch_size]
          y_batch = y[i:i+batch_size]

          # Forward
          output = model(x_batch)
          loss = loss_fn(output, y_batch)

          # Backward
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

      print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item():.4f}")
      torch.save(model.state_dict(), 'MNIST_conv2d.pth')
      # 학습된 모델 저장

else:

  model = torch.load('MNIST_conv2d.pth')


In [ ]:
df2=pd.read_csv('/content/drive/MyDrive/mnist_test.csv')
# 테스트 데이터 로드
pxl = ["pxl{}".format(i) for i in range(len(df.columns)-1)]
df2.columns = ["value"] + pxl
A2 = pd.DataFrame(df2, columns = pxl).values
x_test = torch.tensor(A2, dtype=torch.float32).unsqueeze(1).view(-1,1,28,28)

y_data2 = pd.DataFrame(df2, columns = ["value"]).values
y_data2 = y_data2.reshape(-1)

In [ ]:
# 모델 예측 및 정확도 평가
model.eval()
# eval()로 평가 모드
y_pred = model(x_test).squeeze(1).detach().cpu().numpy()
y_pred = np.argmax(y_pred,axis=1)

In [ ]:
y_pred

In [ ]:
y_data2

In [ ]:
(y_pred == y_data2).sum() / len(y_data2)
# softmax 결과에서 가장 큰 값의 인덱스 = 예측값, 정답과 비교하여 정확도 계산